In [1]:
import pandas as pd 
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
df_preds_lima = pd.read_csv(r"C:\Users\nicol\OneDrive\Master\Goldsmiths\Thesis\weather-prediction\data\df_with_all_preds_lima.csv")

In [3]:
df_preds_lima.head()

Unnamed: 0  temp   dew  humidity  precip  precipprob  precipcover  snow  \
0  2023-05-22  23.1  17.0      70.9     0.1       100.0         4.17   0.0   
1  2023-05-23  23.0  16.9      69.7     0.1       100.0         4.17   0.0   
2  2023-05-24  22.9  17.3      73.6     0.0         0.0         0.00   0.0   
3  2023-05-25  22.9  17.5      70.9     0.1       100.0         4.17   0.0   
4  2023-05-26  22.5  17.8      72.6     0.1       100.0         4.17   0.0   

   snowdepth  windgust  ...  sktime_naive  naive_gs_lima  naive_last  \
0        0.0      13.3  ...     22.300263      19.945401   19.945401   
1        0.0       9.0  ...     22.300526      19.945401   19.945401   
2        0.0      10.8  ...     22.300789      19.945401   19.945401   
3        0.0       9.0  ...     22.301051      19.945401   19.945401   
4        0.0      11.2  ...     22.301314      19.945401   19.945401   

   preds_autoarima_lima  preds_sarimax  lstm_base  lstm_two_layers  \
0             22.238628      22.410406  17.374246        17.674359   
1             22.421975      21.982159  17.278392        17.538429   
2             22.465964      22.125336  17.225514        17.439779   
3             22.381540      22.266274  17.187380        17.364730   
4             22.466933      22.481514  17.154775        17.304440   

   lstm_two_layers_batch_size_1  lstm_features  sarimax_gs  
0                     16.867543      19.670229    0.427177  
1                     16.729293      19.670229    0.427011  
2                     16.571033      19.670229    0.510866  
3                     16.405482      19.670229    0.314001  
4                     16.251267      19.670229    0.276836  

[5 rows x 47 columns]

In [4]:
df_preds_lima.columns

Index(['Unnamed: 0', 'temp', 'dew', 'humidity', 'precip', 'precipprob',
       'precipcover', 'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir',
       'sealevelpressure', 'cloudcover', 'visibility', 'solarradiation',
       'solarenergy', 'uvindex', 'severerisk', 'moonphase',
       'clusters_description', 'year', 'month_1', 'month_2', 'month_3',
       'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9',
       'month_10', 'month_11', 'month_12', 'xgb_base_lima', 'xgb_gs_lima',
       'naive_base_lima', 'sktime_naive', 'naive_gs_lima', 'naive_last',
       'preds_autoarima_lima', 'preds_sarimax', 'lstm_base', 'lstm_two_layers',
       'lstm_two_layers_batch_size_1', 'lstm_features', 'sarimax_gs'],
      dtype='object')

In [6]:
columns_todrop = ['Unnamed: 0', 'dew', 'humidity', 'precip', 'precipprob',
       'precipcover', 'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir',
       'sealevelpressure', 'cloudcover', 'visibility', 'solarradiation',
       'solarenergy', 'uvindex', 'severerisk', 'moonphase',
       'clusters_description', 'year', 'month_1', 'month_2', 'month_3',
       'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9',
       'month_10', 'month_11', 'month_12']
df_preds_lima = df_preds_lima.drop(columns_todrop, axis=1)
df_preds_lima.head()

temp  xgb_base_lima  xgb_gs_lima  naive_base_lima  sktime_naive  \
0  23.1      22.204420    22.007751        19.945401     22.300263   
1  23.0      22.350317    22.109001        19.945401     22.300526   
2  22.9      22.709920    22.018497        19.945401     22.300789   
3  22.9      22.468319    22.425869        19.945401     22.301051   
4  22.5      22.856780    22.813673        19.945401     22.301314   

   naive_gs_lima  naive_last  preds_autoarima_lima  preds_sarimax  lstm_base  \
0      19.945401   19.945401             22.238628      22.410406  17.374246   
1      19.945401   19.945401             22.421975      21.982159  17.278392   
2      19.945401   19.945401             22.465964      22.125336  17.225514   
3      19.945401   19.945401             22.381540      22.266274  17.187380   
4      19.945401   19.945401             22.466933      22.481514  17.154775   

   lstm_two_layers  lstm_two_layers_batch_size_1  lstm_features  sarimax_gs  
0        17.674359                     16.867543      19.670229    0.427177  
1        17.538429                     16.729293      19.670229    0.427011  
2        17.439779                     16.571033      19.670229    0.510866  
3        17.364730                     16.405482      19.670229    0.314001  
4        17.304440                     16.251267      19.670229    0.276836

In [7]:
def calculate_metrics(actual, predicted):
    mae = mean_absolute_error(actual, predicted)
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    r2 = r2_score(actual, predicted)
   #calculate MAPE
    mask = actual != 0  
    mape = np.mean(((actual[mask] - predicted[mask]) / actual[mask])) * 100
    
    return mae, mse, rmse, r2, mape

metrics_list = []

# calculate metrics for each model 
for col in df_preds_lima.columns:
    if col != 'temp':
        mae, mse, rmse, r2, mape = calculate_metrics(df_preds_lima['temp'], df_preds_lima[col])
        metrics_list.append({
            'Model': col,
            'MAE': mae,
            'MSE': mse,
            'RMSE': rmse,
            'R2': r2,
            'MAPE': mape
        })

metrics_df = pd.DataFrame(metrics_list)


In [8]:
metrics_by_mae = metrics_df.sort_values(by='MAE', ascending=True)

In [9]:
metrics_by_mae

Model        MAE         MSE       RMSE         R2  \
1                    xgb_gs_lima   0.671622    0.742997   0.861972   0.845948   
0                  xgb_base_lima   0.798299    0.998735   0.999367   0.792924   
3                   sktime_naive   1.952706    5.189913   2.278138  -0.076069   
6           preds_autoarima_lima   1.987309    5.334147   2.309577  -0.105974   
2                naive_base_lima   2.159706    7.951843   2.819901  -0.648724   
4                  naive_gs_lima   2.159706    7.951843   2.819901  -0.648724   
5                     naive_last   2.159706    7.951843   2.819901  -0.648724   
11                 lstm_features   2.325959    9.001038   3.000173  -0.866262   
8                      lstm_base   5.939040   40.660301   6.376543  -7.430448   
10  lstm_two_layers_batch_size_1   5.946760   40.168629   6.337873  -7.328505   
9                lstm_two_layers   6.230851   43.782136   6.616807  -8.077724   
7                  preds_sarimax   7.251045   88.274745   9.395464 -17.302757   
12                    sarimax_gs  18.063132  331.047419  18.194709 -67.638890   

         MAPE  
1    0.680950  
0    0.860345  
3   -3.947028  
6   -4.322723  
2    7.225741  
4    7.225741  
5    7.225741  
11   8.505681  
8   26.596355  
10  26.662410  
9   28.000059  
7   31.904924  
12  84.215832

In [10]:
#sorted by MAPE

In [11]:
metrics_by_mape = metrics_df.sort_values(by='MAPE', ascending=False)

In [12]:
metrics_by_mape

Model        MAE         MSE       RMSE         R2  \
12                    sarimax_gs  18.063132  331.047419  18.194709 -67.638890   
7                  preds_sarimax   7.251045   88.274745   9.395464 -17.302757   
9                lstm_two_layers   6.230851   43.782136   6.616807  -8.077724   
10  lstm_two_layers_batch_size_1   5.946760   40.168629   6.337873  -7.328505   
8                      lstm_base   5.939040   40.660301   6.376543  -7.430448   
11                 lstm_features   2.325959    9.001038   3.000173  -0.866262   
2                naive_base_lima   2.159706    7.951843   2.819901  -0.648724   
4                  naive_gs_lima   2.159706    7.951843   2.819901  -0.648724   
5                     naive_last   2.159706    7.951843   2.819901  -0.648724   
0                  xgb_base_lima   0.798299    0.998735   0.999367   0.792924   
1                    xgb_gs_lima   0.671622    0.742997   0.861972   0.845948   
3                   sktime_naive   1.952706    5.189913   2.278138  -0.076069   
6           preds_autoarima_lima   1.987309    5.334147   2.309577  -0.105974   

         MAPE  
12  84.215832  
7   31.904924  
9   28.000059  
10  26.662410  
8   26.596355  
11   8.505681  
2    7.225741  
4    7.225741  
5    7.225741  
0    0.860345  
1    0.680950  
3   -3.947028  
6   -4.322723